In [ ]:
# %load_ext autoreload
# %autoreload 2

In [2]:
from acemodel import AceModel, Children
from acemodel import default_mtd, corr_rand_mtd
from spew_data.spew import * 

In [3]:
import brfss
from brfss import bfs_data

In [4]:
# scbrfss = bfs_data('./brfss/SCBRFSS.csv')

In [5]:
data_prex = './spew_data/2010_ver1_45079_synth_'
cdl_data = cdn_data(data_prex)
len(cdl_data)

90314

In [6]:
def simulate_stack(mtd, step_num = 1000, display = False):    

    acemodel_list = [
        AceModel(cdl_data[
            (cdl_data[['_RACE_G1', '_INCOMG']] == [r,i]).all(axis = 1)
        ], r, i, step_method = mtd)
        for r in list(brfss.race_list.keys())[1:]
        for i in list(brfss.income_list.keys())[1:]
                    ]
    
    for i in range(step_num):
        [acemodel.step() for acemodel in acemodel_list]

    result = [res for res in [
            acemodel.datacollector.model_vars['Output'][-1]
            for acemodel in acemodel_list]
                if len(res) > 0
            ]
    
    result_df = pd.concat(result, axis = 0, 
                          ignore_index = True)
    
    if not display:
        return result_df
    
    result_bfs = bfs_data(result_df)
    
    for r in list(brfss.race_list.keys())[1:]:
        for i in list(brfss.income_list.keys())[1:]:
            print('RACE: ' + str(brfss.race_list[r]) + ' INCOME: ' + str(brfss.income_list[i]))
            display(result_bfs.get_corr_mat(r,i))
    return result_df

In [7]:
# scbrfss.get_corr_mat(1,1)

In [8]:
# corr_model = AceModel(cdl_data[
#     (cdl_data[['_RACE_G1', '_INCOMG']] == [1,1]).all(axis = 1)
# ], 1, 1, step_method = corr_rand_mtd)

In [9]:
for i in range(1000):
    corr_model.step()
    
res = corr_model.datacollector.model_vars['Output'][-1]
bfs_data(res).get_prop_mat(1,1)['pr']

29.6 ms ± 352 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
bfs_data(res).get_corr_mat(1,1)

,depress,alcoabuse,drugabuse,prison,patdivorce,phyabuse1,phyabuse2,verbalabuse,sexabuse1,sexabuse2,sexabuse3,foodinsecure
depress,1.000000,0.010122,0.049598,-0.000883,0.053106,0.050111,0.011549,0.017128,0.014250,0.051108,-0.006749,0.032197
alcoabuse,0.010122,1.000000,0.026782,0.032613,0.053480,0.048974,0.059274,0.042515,0.028561,0.025361,0.015398,0.026815
drugabuse,0.049598,0.026782,1.000000,0.016165,-0.006042,0.023128,0.015945,0.024960,0.033851,0.025083,0.005760,0.013595
prison,-0.000883,0.032613,0.016165,1.000000,0.022753,-0.020575,-0.012685,-0.034721,-0.008762,0.031679,0.009474,-0.005194
patdivorce,0.053106,0.053480,-0.006042,0.022753,1.000000,0.032629,0.032287,0.005931,0.011336,0.033828,0.029735,0.054761
phyabuse1,0.050111,0.048974,0.023128,-0.020575,0.032629,1.000000,0.093472,0.049506,0.123091,0.076894,0.049631,0.031875
phyabuse2,0.011549,0.059274,0.015945,-0.012685,0.032287,0.093472,1.000000,0.018347,0.063479,0.078519,0.061377,0.059953
verbalabuse,0.017128,0.042515,0.024960,-0.034721,0.005931,0.049506,0.018347,1.000000,0.025276,0.080544,0.034233,0.037130
sexabuse1,0.014250,0.028561,0.033851,-0.008762,0.011336,0.123091,0.063479,0.025276,1.000000,0.102026,0.092701,0.007716
sexabuse2,0.051108,0.025361,0.025083,0.031679,0.033828,0.076894,0.078519,0.080544,0.102026,1.000000,0.057863,0.030279


In [13]:
default_result = simulate_stack(mtd = default_mtd, display = False)

In [14]:
corr_result = simulate_stack(mtd = corr_rand_mtd, display = False)